In [1]:
import sqlalchemy as sql
import pandas as pd
import glob
from os import path, sep
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftshift
from scipy.signal import oaconvolve
from scipy.signal import remez
import time
import decimate as d
from importlib import reload
reload(d);

In [2]:
time.perf_counter()

10559.873378209

In [3]:
f3 = remez(4000,[0., 0.5, 1., 250.],[1.,0.],fs=500)


In [4]:
server = "thesis.ca6j6heoraog.eu-central-1.rds.amazonaws.com"
engine = sql.create_engine(f"mysql+pymysql://admin:FphvsYQek4@{server}/thesis_db")

In [5]:
con = engine.connect()

```SQL
create table interpolation_types (
	id int(11) NOT NULL PRIMARY KEY AUTO_INCREMENT,
    interp_algo varchar(45),
    algo_version varchar(20),
    notebook_name varchar(45),
    sampling_rate float(6,4)
    )
```

In [7]:

stmnt ="""
INSERT IGNORE INTO interpolation_series
values 
    (1, "legacy", "", NULL, 0.5),
    (2, "piecewise", "ec31c78", "interpolate and upload", 0.25),
    (3, "piecewise", "716f804", "interpolate and upload", 0.25)
"""

con.execute(stmnt)

/home/siegman/miniconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry '1' for key 'PRIMARY'")
  result = self._query(query)
/home/siegman/miniconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry '2' for key 'PRIMARY'")
  result = self._query(query)


In [8]:
stmnt = """
SELECT DISTINCT series_type_id, group_id, participant_id
FROM raw_interval_data
"""

raw_series = con.execute(stmnt)

In [9]:
raw_series

In [10]:
get_series = sql.text("""
SELECT data_time
FROM raw_interval_data
WHERE series_type_id = :sti
AND group_id = :gi
AND participant_id = :pi""")

In [11]:
tref = time.perf_counter()
for series_type_id, group_id, participant_id in raw_series:
    print(series_type_id, group_id, participant_id, np.around(time.perf_counter()-tref,2) ) 
    q = con.execute(get_series, sti=series_type_id, gi= group_id, pi=participant_id)
    t = np.array(q.fetchall()).astype(float).squeeze()
    x = t[:-1] # don't use the point which marks the end of the experiment
    y = np.diff(t)
    x_pw = np.arange(0,t[-1],0.5)
    y_pw = d.pw_constant_convolve(x, y, t[-1], f3, 4, 3, x_pw)
    nx = len(x_pw)
    interpolated_data = pd.DataFrame({
        'series_type_id': [series_type_id]*nx,
        'interpolation_series_id': [3]*nx,
        'group_id': [group_id]*nx,
        'participant_id': [participant_id]*nx,
        'data_ordinal': range(nx),
        'data_time': x_pw,
        'data_value': np.around(y_pw,5)
    })
    interpolated_data.to_sql(name='interpolated_interval_data', con=con, if_exists='append', index=False, method='multi')   


1 1001 101 0.0
1 1001 102 0.46
1 1001 103 0.86
1 1003 108 1.26
1 1003 109 1.66
1 1004 110 2.04
1 1004 111 2.4
1 1004 112 2.78
1 1005 113 3.15
1 1005 114 3.52
1 1005 115 3.9
1 1006 116 4.27
1 1006 117 4.68
1 1006 118 5.07
1 1007 119 5.44
1 1007 120 5.83
1 1007 121 6.2
1 1008 122 6.59
1 1008 123 6.93
1 1008 124 7.27
1 1009 125 7.73
1 1009 126 8.11
1 1009 127 8.48
1 1010 128 8.85
1 1010 129 9.23
1 1010 130 9.59
1 1011 131 9.97
1 1011 132 10.35
1 1011 133 10.76
1 1012 134 11.11
1 1012 135 11.49
1 1012 136 11.87
1 1013 137 12.21
1 1013 138 12.58
1 1013 139 12.96
1 1014 140 13.37
1 1014 141 13.72
1 1014 142 14.05
1 1015 144 14.38
1 1015 145 14.71
1 1016 146 15.02
1 1016 147 15.38
1 1016 148 15.76
1 1017 149 16.21
1 1017 151 16.58
1 1019 155 16.9
1 1019 156 17.28
1 1019 157 17.68
1 1020 158 18.06
1 1020 159 18.42
1 1020 160 18.77
1 1022 164 19.12
1 1022 165 19.48
1 1022 166 19.86
1 1023 167 20.21
1 1023 168 20.57
1 1023 169 20.93
1 1024 170 21.28
1 1024 171 21.69
1 1024 172 22.05
1 1025 173 2

In [ ]:
y_pw = d.pw_constant_convolve(x, y, t[-1], f3, 4, 3, x_pw)
nx = len(x_pw)
interpolated_data = pd.DataFrame({
    'series_type_id': [series_type_id]*nx,
    'group_id': [group_id]*nx,
    'participant_id': [participant_id]*nx,
    'data_ordinal': range(nx),
    'data_time': x_pw,
    'data_value': np.around(y_pw,5)
})
interpolated_data.to_sql(name='interpolated_interval_data', con=con, if_exists='append', index=False, method='multi')   


In [ ]:
df

In [ ]:
(t[-1], f3, 4, 3, x_pw)

In [ ]:
x

In [ ]:
y